In [1]:
import os
import numpy
def permutation(lst): 
  
    # If lst is empty then there are no permutations 
    if len(lst) == 0: 
        return [] 
  
    # If there is only one element in lst then, only 
    # one permuatation is possible 
    if len(lst) == 1: 
        return [lst] 
  
    # Find the permutations for lst if there are 
    # more than 1 characters 
  
    l = [] # empty list that will store current permutation 
  
    # Iterate the input(lst) and calculate the permutation 
    for i in range(len(lst)): 
        m = lst[i] 
  
       # Extract lst[i] or m from the list.  remLst is 
       # remaining list 
        remLst = lst[:i] + lst[i+1:] 
  
       # Generating all permutations where m is first 
       # element 
        for p in permutation(remLst): 
            l.append([m] + p) 
    return l 
def file_slice(filepath, start_target, end_target, start_offset = 0, end_offset=0):
    infile = open(filepath,'r')
    lines = infile.readlines()
    infile.close()
    start = 0
    end = 0
    counter = 0
    for line in lines:
        if line == start_target:
            start = counter
        if line == end_target:
            end = counter
        counter += 1
    start += start_offset
    if end != 0:
        end += end_offset
    slice = lines[start:end]
    return(slice, start, end)
def translate(dictionary, list):
    translated = []
    for element in list:
        element = dictionary[element]
        translated.append(element)
    return(translated)
def update_xml(xml_path, itp_path):
    outfile = open(itp_path,'r')
    data = outfile.readlines()
    outfile.close()
    file = itp_path
    dictdata, startline, endline = file_slice(file,'; nr type resnr residue atom cgnr charge\n','[ bonds ]\n', start_offset = 1, end_offset = -1)
    master_dict = {}
    masses = {}
    charges = {}
    for line in dictdata:
        line = line.split(" ")
        master_dict[line[0]] = line[1]
        charges[line[1]] = line[-1]
    bondlist, startline, endline = file_slice(file,'[ bonds ]\n','[ angles ]\n', start_offset = 2, end_offset = -1)
    bond_params = []
    if endline != 0:
        translated_bonds = []
        for bond in bondlist:
            bond = bond.split(' ')
            [bond[0],bond[1]] = translate(master_dict, [bond[0],bond[1]])
            s = ' '
            bond = s.join(bond)
            translated_bonds.append(bond)
            if bond not in bond_params:
                bond_params.append(bond)

    anglelist, startline, endline = file_slice(file,'[ angles ]\n','[ dihedrals ]\n', start_offset = 2, end_offset = -1)
    angle_params = []
    if endline != 0:
        translated_angles = []
        for angle in anglelist:
            angle = angle.split(' ')
            [angle[0],angle[1],angle[2]] = translate(master_dict,[angle[0],angle[1],angle[2]])
            s = ' '
            angle = s.join(angle)
            translated_angles.append(angle)
            if angle not in angle_params:
                angle_params.append(angle)
    dihedrallist, startline, endline = file_slice(file,'[ dihedrals ]\n','[ dihedrals ] ; impropers\n', start_offset = 2, end_offset = -1)
    dihedral_params = []
    if endline != 0:
        translated_dihedrals = []
        for dihedral in dihedrallist:
            dihedral = dihedral.split(' ')
            [dihedral[0],dihedral[1],dihedral[2],dihedral[3]] = translate(master_dict, [dihedral[0],dihedral[1],dihedral[2],dihedral[3]])
            s = ' '
            dihedral = s.join(dihedral)
            translated_dihedrals.append(dihedral)
            if dihedral not in dihedral_params:
                dihedral_params.append(dihedral)
    improperlist, startline, endline = file_slice(file,'[ dihedrals ] ; impropers\n','[ pairs ]\n', start_offset = 2, end_offset = -1)
    improper_params = []
    if endline != 0:
        translated_impropers = []
        for improper in improperlist:
            improper = improper.split(' ')
            [improper[0],improper[1],improper[2],improper[3]] = translate(master_dict, [improper[0],improper[1],improper[2],improper[3]])
            s = ' '
            improper = s.join(improper)
            translated_impropers.append(improper)
            if improper not in improper_params:
                improper_params.append(improper)
    atoms = []
    for atom in master_dict.values():
        if atom not in atoms:
            atoms.append(atom)
    master = open('foyer.xml','w+')
    # convert things into correct format to add to master list
    xml_bonds = []

    for bond in bond_params:
        bond = bond.strip('\n')
        elements = bond.split(' ')
        xml_bond = F'    <Bond type1="{elements[0]}" type2="{elements[1]}" length="{elements[3]}" k="{elements[-1]}"/>\n'
        xml_bonds.append(xml_bond)
    existing_bonds, startline, endline = file_slice(xml_path,'  <HarmonicBondForce>\n','  </HarmonicBondForce>\n', start_offset = 1)
    master_data = open(xml_path,'r').readlines()
    for i in range(startline):
        master.write(master_data[i])
    for bond in existing_bonds:
        master.write(bond)
    existing_bondtypes = []
    for bond in existing_bonds:
        bond = bond.split('"')
        for bondtype in permutation([bond[1],bond[3]]):
            existing_bondtypes.append(bondtype)
    out = open('bond_add.txt','w+')
    for bond in xml_bonds:
        bond1 = bond.split('"')
        bondtype = [bond1[1], bond1[3]]
        if bondtype not in existing_bondtypes:
            out.write(bond)
            master.write(bond)
    out.close()
    xml_angles = []
    track = endline
    for angle in angle_params:
        angle = angle.strip('\n')
        elements = angle.split(' ')
        xml_angle = F'    <Angle type1="{elements[0]}" type2="{elements[1]}" type3="{elements[2]}" angle="{elements[4]}" k="{elements[-1]}"/>\n'
        xml_angles.append(xml_angle)
    existing_angles, startline, endline = file_slice(xml_path,'  <HarmonicAngleForce>\n','  <HarmonicAngleForce>\n', start_offset = 1)
    master_data = open(xml_path,'r').readlines()
    for i in range(startline-track):
        master.write(master_data[i+track])
    for angle in existing_angles:
        master.write(angle)
    existing_angletypes = []
    for angle in existing_angles:
        angle = angle.split('"')
        for existing in permutation([angle[1],angle[3],angle[5]]):
            existing_angletypes.append(existing)
    out = open('angle_add.txt','w+')
    for angle in xml_angles:
        angle1 = angle.split('"')
        type = [angle1[1], angle1[3], angle1[5]]
        if type not in existing_angletypes:
            out.write(angle)
            master.write(angle)
    out.close()
    track = endline  
    xml_di = []
    for di in dihedral_params:
        elements = di.split(' ')
        period = elements[-1].strip()
        rad = str(numpy.radians(float(elements[5])))
        k1 = elements[6]
        xml_d = F'    <Proper type1="{elements[0]}" type2="{elements[1]}" type3="{elements[2]}" type4="{elements[3]}" periodicity1="{period}" phase1="{rad}" k1="{k1}"/>\n'
        xml_di.append(xml_d)
    master_data = open(xml_path,'r').readlines()
    existing_di, startline, endline = file_slice(xml_path,'  <PeriodicTorsionForce>\n','  </PeriodicTorsionForce>\n', start_offset = 1)
    for i in range(startline-track):
        master.write(master_data[i+track])
    for di in existing_di:
        master.write(di)
    existing_ditypes = []
    for di in existing_di:
        di = di.split('"')
        for existing in permutation([di[1], di[3], di[5], di[7]]):
            existing_ditypes.append(existing)   

    out = open('proper_add.txt','w+')
    for di in xml_di:
        di1 = di.split('"')
        type = [di1[1], di1[3], di1[5], di1[7]]
        if type not in existing_ditypes:
            out.write(di)
            master.write(di)
    out.close()
    xml_im = []
    track = endline
    for im in improper_params:
        elements = im.strip().split(' ')
        rad = str(numpy.radians(float(elements[5])))
        xml_i = F'    <Improper type1="{elements[0]}" type2="{elements[1]}" type3="{elements[2]}" type4="{elements[3]}" k="{elements[6]}" theta0="{rad}"/>\n'
        xml_im.append(xml_i)
    master_data = open(xml_path,'r').readlines()
    existing_im, startline, endline = file_slice(xml_path,'  <CustomTorsionForce energy="k*(theta-theta0)^2">\n','  </CustomTorsionForce>\n', start_offset = 3)
    for i in range(startline-track):
        master.write(master_data[i+track])
    for im in existing_im:
        master.write(im)
    existing_imtypes = []
    for im in existing_im:
        im = im.split('"')
        for existing in permutation([im[1], im[3], im[5], im[7]]):
            existing_imtypes.append(existing)       
    out = open('improper_add.txt','w+')
    for im in xml_im:
        im1 = im.split('"')
        type = [im1[1], im1[3], im1[5], im1[7]]
        if type not in existing_imtypes:
            out.write(im)
            master.write(im)
    out.close()
    masterfile = open(xml_path,'r')
    lines = masterfile.readlines()
    masterfile.close()
    f = False
    for line in lines:
        if line == '  </CustomTorsionForce>\n':
            f = True
        if f:
            master.write(line)
    master.close()
    output = open('translated.txt','w+')
    output.write('<atomtypes>\n')
    for atom in atoms:
        output.write(F'{atom}\n')
    output.write('<atomtypes>\n')
    output.write('\n')
    output.write('<bonds>\n')

    for bond in bond_params:
        output.write(F'{bond}')
    output.write('<bonds>\n')
    output.write('\n')
    output.write('<angles>\n')

    for angle in angle_params:
        output.write(F'{angle}')
    output.write('<angles>\n')
    output.write('\n')
    output.write('<dihedrals>\n')

    for dihedral in dihedral_params:
        output.write(F'{dihedral}')
    output.write('<dihedrals>\n')
    output.write('\n')
    output.write('<impropers>\n')

    for improper in improper_params:
        output.write(F'{improper}')
    output.write('<impropers>\n')
    output.close()

In [2]:
update_xml('foyer_charmm.xml', os.path.join("atomistic","tip3p_pppm","tip3p.itp"))
if os.path.exists('foyer_backup.xml'):
    os.remove('foyer_backup.xml')
os.rename("foyer_charmm.xml","foyer_backup.xml")
os.rename("foyer.xml","foyer_charmm.xml")